<a href="https://colab.research.google.com/github/BaptisteZloch/Birds-species-spotting/blob/main/ALBATROSS_vs_CANARY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle API installing

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload() #upload your JSON token Kaggle API

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

# Dataset


In [ ]:
!kaggle datasets download -d gpiosenka/100-bird-species #files loading

In [ ]:
!unzip 100-bird-species.zip #files extracting

In [ ]:
!cp -R /content/valid/ALBATROSS/ /content/Valid_2/
!cp -R /content/valid/CANARY/ /content/Valid_2/
!cp -R /content/train/ALBATROSS/ /content/Train_2/
!cp -R /content/train/CANARY/ /content/Train_2/

## Image preprocessing

We will label the data based on their folders and we will also use data augmentation to make our model better.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be augmented
train_datagen = ImageDataGenerator(
                  rotation_range=40,
                  width_shift_range=0.2,
                  height_shift_range=0.2,
                  shear_range=0.2,
                  zoom_range=0.2,
                  horizontal_flip=True,
                  fill_mode='nearest',
                  rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/content/Train_2/',  # This is the source directory for training images
        target_size=(128, 128),
        batch_size=128,
        class_mode='binary',
        color_mode='rgb')

validation_datagen = ImageDataGenerator(rescale=1/255)

validation_generator = validation_datagen.flow_from_directory(
        '/content/Valid_2/',
        target_size=(128, 128),
        class_mode='binary',
        color_mode='rgb')

# Model

## Build the model

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),
             loss='binary_crossentropy',
             metrics=['accuracy'])


model.summary()

## Train the model

In [ ]:
history = model.fit(
     train_generator,
      steps_per_epoch= 293 // 128,
      epochs=15,
      verbose=1,
      validation_data=validation_generator)

In [ ]:
model.save("mymodel.h5")

## Evalute the model

In [ ]:
import numpy as np
from keras.preprocessing import image

species = ['CANARY','ALBATROSS']

for specie in species:
  for i in range(1,6):
    img = image.load_img('/content/test/'+specie+'/'+ str(i) +'.jpg', target_size=(128, 128))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    image_tensor = np.vstack([x])
    classes = model.predict(image_tensor)
    if classes[0]>0.5:
      print("Predicted  : CANARY      Real : "+specie)
    else:
      print("Predicted  : ALBATROSS      Real : "+specie)
  